In [1]:
from transformers import MobileNetV2ForImageClassification, BasicTokenizer
from torch.utils.data import DataLoader
from datasets import load_from_disk
import torch.nn as nn
import torch
import numpy as np
import base

In [4]:
base.count_parameters(model)

model size: 24.918MB.
Total Trainable Params: 1686002.


,Modules,Parameters
0,lstm.weight_ih_l0,360000
1,lstm.weight_hh_l0,360000
2,lstm.bias_ih_l0,1200
3,lstm.bias_hh_l0,1200
4,lstm.weight_ih_l0_reverse,360000
5,lstm.weight_hh_l0_reverse,360000
6,lstm.bias_ih_l0_reverse,1200
7,lstm.bias_hh_l0_reverse,1200
8,fc1.weight,240000
9,fc1.bias,400


In [9]:
def prep_data_experimental(dataset, padding, tokenizer, GLOVE_FILE): 

    dataset_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), dataset))
    all_tokens = []
    for data in dataset:
        for token in data:
            all_tokens.append(token)
    vocab = set(all_tokens)
    vocab_size = len(vocab)+2

    word_index = dict(zip(vocab, range(len(vocab))))

    embeddings_index = {}
    with open(GLOVE_FILE, encoding='utf-8') as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embeddings_index[word] = coefs

    embedding_matrix = np.zeros((vocab_size, 300))

    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    embedding_matrix = torch.tensor(embedding_matrix, dtype=torch.float32)

    dataset_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),dataset_tokens))
    dataset_padded = list(map(lambda x: padd(x,padding), dataset_index))
    dataset = dataset.add_column("input_ids", dataset_padded)
    return dataset, embedding_matrix


def padd(data, max_length):
    padding_length = max_length - len(data)
    if padding_length > 0:
        padding = [0 for _ in range(padding_length)]
        data.extend(padding)
    return data[:max_length]

In [ ]:
tokenizer = BasicTokenizer(do_lower_case=True)
test_data = load_from_disk("./data/sst2/eval-logits")

test_data, embedding_matrix = prep_data_experimental(test_data, 60, tokenizer, "/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1/glove.6B.300d.txt") 

In [ ]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=300, fc_dim=400, hidden_dim=300, output_dim=2)
model.load_state_dict(torch.load("models/sst2/bilstm-distill-aug.pth"))
model.eval()

In [ ]:
torch.cuda.synchronize() 
starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
device = "cuda"
model.to(device)

test_loader = DataLoader(test_data, batch_size=1, shuffle=False)
timings = []



for i, batch in enumerate(test_loader):
    if i >= 1000:
        break
    torch.cuda.synchronize()
    starter.record()
    with torch.no_grad():
        _ = model(batch["input_ids"].to(device))
    ender.record()
    torch.cuda.synchronize()
    timings.append(starter.elapsed_time(ender))

print(f"Average Inference Time on GPU: {sum(timings) / len(timings):.3f} ms")

In [ ]:
from torch.utils.data import DataLoader
import time

test_loader = DataLoader(test, batch_size=1, shuffle=False)
timings = []
device = "cpu"
model.to(device)

for i, batch in enumerate(test_loader):
    if i >= 1000:
        break
    start_time = time.perf_counter()
    with torch.no_grad():
        _ = model(batch["input_ids"].to(device))
    end_time = time.perf_counter()
    timings.append((end_time - start_time)*1000)


print(f"Average Inference Time on CPU: {sum(timings) / len(timings):.3f} ms")

In [2]:
model_pretrained = MobileNetV2ForImageClassification.from_pretrained("google/mobilenet_v2_1.0_224")

In [3]:
base.count_parameters(model_pretrained)

model size: 13.505MB.
Total Trainable Params: 3506153.


,Modules,Parameters
0,mobilenet_v2.conv_stem.first_conv.convolution....,864
1,mobilenet_v2.conv_stem.first_conv.normalizatio...,32
2,mobilenet_v2.conv_stem.first_conv.normalizatio...,32
3,mobilenet_v2.conv_stem.conv_3x3.convolution.we...,288
4,mobilenet_v2.conv_stem.conv_3x3.normalization....,32
...,...,...
153,mobilenet_v2.conv_1x1.convolution.weight,409600
154,mobilenet_v2.conv_1x1.normalization.weight,1280
155,mobilenet_v2.conv_1x1.normalization.bias,1280
156,classifier.weight,1281280


In [4]:
in_features = model_pretrained.classifier.in_features
model_pretrained.classifier = nn.Linear(in_features,100) #Úprava klasifikační hlavy
model_pretrained.num_labels = 100
model_pretrained.config.num_labels = 100

In [5]:
base.count_parameters(model_pretrained)

model size: 9.103MB.
Total Trainable Params: 2351972.


,Modules,Parameters
0,mobilenet_v2.conv_stem.first_conv.convolution....,864
1,mobilenet_v2.conv_stem.first_conv.normalizatio...,32
2,mobilenet_v2.conv_stem.first_conv.normalizatio...,32
3,mobilenet_v2.conv_stem.conv_3x3.convolution.we...,288
4,mobilenet_v2.conv_stem.conv_3x3.normalization....,32
...,...,...
153,mobilenet_v2.conv_1x1.convolution.weight,409600
154,mobilenet_v2.conv_1x1.normalization.weight,1280
155,mobilenet_v2.conv_1x1.normalization.bias,1280
156,classifier.weight,128000


In [6]:
in_features = model_pretrained.classifier.in_features
model_pretrained.classifier = nn.Linear(in_features,10) #Úprava klasifikační hlavy
model_pretrained.num_labels = 10
model_pretrained.config.num_labels = 10

In [7]:
base.count_parameters(model_pretrained)

model size: 8.663MB.
Total Trainable Params: 2236682.


,Modules,Parameters
0,mobilenet_v2.conv_stem.first_conv.convolution....,864
1,mobilenet_v2.conv_stem.first_conv.normalizatio...,32
2,mobilenet_v2.conv_stem.first_conv.normalizatio...,32
3,mobilenet_v2.conv_stem.conv_3x3.convolution.we...,288
4,mobilenet_v2.conv_stem.conv_3x3.normalization....,32
...,...,...
153,mobilenet_v2.conv_1x1.convolution.weight,409600
154,mobilenet_v2.conv_1x1.normalization.weight,1280
155,mobilenet_v2.conv_1x1.normalization.bias,1280
156,classifier.weight,12800


In [ ]:
from importlib import reload
import base
reload(base)
import base

In [2]:
import torch
torch.version.cuda

'12.6'